In [85]:
import pandas as pd

# Load the Brands.xlsx file and create a list of brands
brands= pd.read_excel('Brands.xlsx')

# Change the excel file to csv
sales_curr=pd.read_excel ('SalesReport_2022_04.xlsx')
sales_curr.to_csv ('SalesReport_2022_04.csv', 
                  index = None,
                  header=True)

In [86]:
# Load the LeadReport and SalesReport files for the previous month and current month
lead_prev_month= pd.read_csv('LeadReport_2022_03.csv', delimiter= ';')
lead_curr_month = pd.read_csv('LeadReport_2022_04.csv', delimiter= ';')
sales_prev_month=pd.read_csv('SalesReport_2022_03.csv', error_bad_lines=False, delimiter= ';')
sales_curr_month=pd.read_csv('SalesReport_2022_04.csv', delimiter= ';')

In [87]:
# Merge lead and sales data for current month
data_curr = pd.merge(lead_curr_month, sales_curr_month, on=['Brand Name'])

data_curr['My Sale'] = data_curr['My Sale'].replace (to_replace = ['yes','no'],value = ['1','0']) 
data_curr['My Sale']  = pd.to_numeric(data_curr['My Sale'] )

# Group by brand and sum the leads and sales
data_curr = data_curr.groupby('Brand Name').agg({'Lead ID': 'nunique', 'My Sale': 'sum'}).reset_index()

# Rename columns
data_curr.columns = ['Brand', 'Leads Current Month', 'Sales Current Month']

In [88]:
# Merge lead and sales data for previous month
data_prev = pd.merge(lead_prev_month, sales_prev_month, on=['Brand Name'])
data_prev['My Sale'] = data_prev['My Sale'].replace (to_replace = ['yes','no'],value = ['1','0']) 
data_prev['My Sale']  = pd.to_numeric(data_prev['My Sale'] )

# Group by brand and sum the leads and sales
data_prev = data_prev.groupby('Brand Name').agg({'Lead ID': 'nunique', 'My Sale': 'sum'}).reset_index()

# Rename columns
data_prev.columns = ['Brand', 'Leads Previous Month', 'Sales Previous Month']

In [89]:
# Merge previous and current month data
data_kpi = pd.merge(data_prev, data_curr, on='Brand')


In [90]:
# Convert list to DataFrame
data_kpi = pd.DataFrame(brands)

# Merge previous and current month data
data_kpi = pd.merge(data_prev, data_curr, on='Brand')


In [91]:

# Calculate conversion rate
data_kpi['Conversion Previous Month'] = data_kpi['Sales Previous Month'] / data_kpi['Leads Previous Month']
data_kpi['Conversion Current Month'] = data_kpi['Sales Current Month'] / data_kpi['Leads Current Month']

# Calculate conversion growth %
data_kpi['Conversion Growth %'] =round((data_kpi['Conversion Current Month'] - data_kpi['Conversion Previous Month']) / data_kpi['Conversion Previous Month'] * 100,2)
# Merge with brands list
data_kpi = pd.merge(data_kpi, brands, on='Brand', how='left')


In [92]:
import openpyxl 
from openpyxl import Workbook

# Save KPIs to Excel file
wb = Workbook()
ws = wb.active

# Write headers
ws.append(['Date', 'Brand', 'Leads Previous Month', 'Leads Current Month', 'Sales Previous Month', 'Sales Current Month', 'Conversion Previous Month', 'Conversion Current Month', 'Conversion Growth %'])

# Write data
for index, row in data_kpi.iterrows():
    ws.append(['2022-04-01', row['Brand'], row['Leads Previous Month'], row['Leads Current Month'], row['Sales Previous Month'], row['Sales Current Month'], row['Conversion Previous Month'], row['Conversion Current Month'],row['Conversion Growth %']])

In [93]:
# Save file
wb.save('2022_04 KPIs.xlsx')

In [94]:
pd.read_excel('2022_04 KPIs.xlsx')

,Date,Brand,Leads Previous Month,Leads Current Month,Sales Previous Month,Sales Current Month,Conversion Previous Month,Conversion Current Month,Conversion Growth %
0,2022-04-01,ASUS,613,632,4291,2528,7,4,-42.86
1,2022-04-01,Acer,165,155,165,155,1,1,0.00
2,2022-04-01,Asus ROG,124,75,124,75,1,1,0.00
3,2022-04-01,De'Longhi,48,41,144,0,3,0,-100.00
4,2022-04-01,Kenwood,39,36,117,0,3,0,-100.00
5,2022-04-01,MSI Systems,551,315,551,315,1,1,0.00
6,2022-04-01,One For All,266,227,2128,454,8,2,-75.00
7,2022-04-01,Razer,314,704,1884,6336,6,9,50.00
8,2022-04-01,Samsung Global,5242,7380,283068,428040,54,58,7.41
9,2022-04-01,Sony,12618,9648,504720,318384,40,33,-17.50
